### Middeleware
##### Middleware provides a way to more tightly control what happens inside the agent. Middleware is useful for the following:
* Tracking agent behavior with logging,analytics, and debugging
* Transforming prompts,tool selection, and output formatting
* Adding retries, fallbacks, and early termination logic
* Applying rate limits, guardrails and PII detection
##### it can be model check, exception checking, etc., etc.
##### Agent with Middleware --> we can add some hooks

### Built-in Middleware (Ref: https://docs.langchain.com/oss/python/langchain/middleware/built-in#built-in-middleware)
* 1. Summarization ===> we add this middleware, after some messages it will summarize. This summarization of middleware can be taken care by LLM
* 2. Human-in-the-loop ===> 
* 3. Model call limit
* 4. Tool call limit 
* 5. PII detection

In [1]:
# Import keys 
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


### Summarization Middleware
##### Automatically summarize conversation history when approaching token limits, preserving recent messages while compressing older context. Summarization is useful for the following:
* Long-running conversations that exceed context windows.
* Multi-turn dialogues with extensive history.
* Applications where preserving full conversation context matters.


In [5]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage,SystemMessage


agent = create_agent(
    model="gpt-4o-mini",
    tools=[], # empty as not defined yet
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(     # Applying Summarization within our agent
            model="gpt-4o-mini",     # What LLM model to use for summarization? Here, gpt-4o-mini
            trigger=("tokens",10),   # when messages length become 10
            keep=("messages",4)     # keep the top 4 messages while preserving
                                    # add any number of middleware techniques as required 
        ),
    ],
)

In [6]:
# Before invoke this agent, create a thread 
# Run with thread id 
config={"configurable":{"thread_id":"test-1"}}

In [9]:
# Alternative test data 
questions=[
    "What is 3+5?",
    "What is 10-4?",
    "What is 6*7?",
    "What is 18+9?",
    "What is 45-17?",
    "What is 8*9?",
    "What is 100+25?",
    "What is 64/8?",
    "What is 12*4?",
    "What is 90-36?"
]


for q in questions:
    response=agent.invoke({"messages":[HumanMessage(content=q)]},config)
    print(f"Messages:{response}")
    print(f"Messages:{len(response['messages'])}")


Messages:{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nWhat is 2+2? -> 2 + 2 equals 4.  \nWhat is 2-2? -> 2 - 2 equals 0.  \nWhat is 2*2? -> 2 * 2 equals 4.  \nWhat is 20+2? -> 20 + 2 equals 22.  \nWhat is 22+22? -> 22 + 22 equals 44.  \nWhat is 200+2? -> 200 + 2 equals 202.  \nWhat is 2000*2? -> 2000 * 2 equals 4000.', additional_kwargs={}, response_metadata={}, id='56cf36cd-f806-4e05-9986-71bb7217dabc'), AIMessage(content='200 + 2 equals 202.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 191, 'total_tokens': 199, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-Crajig6MH5E10HyZaOxGe1HlCmjgT', 'service_

### token size

In [11]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.tools import tool

@tool
def search_hotels(city:str)->str:
    """search tokens - returns long response to use more tokens"""
    return f"""Hotels in {city}:
    1. Grand Hotel - 5 star, $350/night, spa, pool, gym
    2. City Inn - 4 star, $180/night, business center 
    3. Budget Stay - 3 star, $75/night, free wifi"""

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_hotels], # empty as not defined yet
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(      # Applying Summarization within our agent
            model="gpt-4o-mini",      # What LLM model to use for summarization? Here, gpt-4o-mini
            trigger=("tokens",500),   # when messages length become 10
            keep=("tokens",200)      # keep the top 4 messages while preserving
                                     # add any number of middleware techniques as required 
        ),
    ]
)

#Create Config
config={"configurable":{"thread_id":"test-1"}}

# Token counter (approximate)
def count_tokens(messages):
    total_chars = sum(len(str(m.content)) for m in messages)
    return total_chars // 4  # 4 chars = 1 token


In [12]:
# Run test 
cities=["Paris","NYC","London","Dubai","Mumbai","Milan","Zurich"]

for city in cities:
    response=agent.invoke(
        {"messages":[HumanMessage(content=f"Find the hotel in {city}")]},
        config=config
    )

tokens=count_tokens(response["messages"])
print(f"{city}:~{tokens} tokens, {len(response['messages'])} messages")
print(f"{(response['messages'])}")

Zurich:~573 tokens, 6 messages
[HumanMessage(content='Here is a summary of the conversation to date:\n\nUser requested hotel information for several cities: Paris, NYC, London, Dubai, Mumbai, and Milan. The AI provided hotel lists with details on star ratings, prices, and amenities for each city:\n\n**Hotels in Paris:**\n1. Grand Hotel - 5 stars, $350/night, Amenities: Spa, pool, gym\n2. City Inn - 4 stars, $180/night, Amenities: Business center\n3. Budget Stay - 3 stars, $75/night, Amenities: Free wifi\n\n**Hotels in NYC:**\n1. Grand Hotel - 5 stars, $350/night, Amenities: Spa, pool, gym\n2. City Inn - 4 stars, $180/night, Amenities: Business center\n3. Budget Stay - 3 stars, $75/night, Amenities: Free wifi\n\n**Hotels in London:**\n1. Grand Hotel - 5 stars, $350/night, Amenities: Spa, pool, gym\n2. City Inn - 4 stars, $180/night, Amenities: Business center\n3. Budget Stay - 3 stars, $75/night, Amenities: Free wifi\n\n**Hotels in Dubai:**\n1. Grand Hotel - 5 stars, $350/night, Ameniti

### Human-in-the-loop Middleware
##### Pause agent execution for human approval, editing, or rejection of tool calls before they execute. Human-in-the-loop is useful for the following:
* High-stakes operations requiring human approval (eg. database writes,financial transactions)
* Compliance workflows where human oversight is mandatory
* Long-running conversations where human feedback guides the agent

In [16]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id:str) -> str:
    """Sample function to read an email by its ID."""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient:str,subject:str,body:str) -> str:
    """Sample function to send an email."""
    return f"Email send to {recipient} with subject '{subject}'"

In [23]:
agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email_tool,send_email_tool], 
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approved","edit","reject"]
                },
                "read_email_tool":False,
            }      
            
        ),
    ]
)

In [19]:
config={"configurable":{"thread_id":"test-approve"}}

#Step1: Request 
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to abc@example.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

In [20]:
result

{'messages': [HumanMessage(content="Send email to abc@example.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='dc08edcb-93c6-4ed2-be72-2eabebf1f153'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 97, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bbc38b4db', 'id': 'chatcmpl-Crbagat8JYbgaEf7Clr67QcX0jRDO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b62f7-13bd-7093-86c6-2ffcd400bfd3-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'abc@example.com', 'subject': 'Hello', 'body': 'How are you?'}, 'id': 'call_eUXyzCe

In [27]:
# Approve process 
from langgraph.types import Command 

if "__interrupt__" in result:
    print(" Paused ! Approving...")

result = agent.invoke(
    Command(
        resume={
            "decisions":[{"type":"approve"}],
            "messages": result["messages"]
        }
    ),
    config=config
)

print(f"Result: {result['messages'][-1].content}")

 Paused ! Approving...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages': empty array. Expected an array with minimum length 1, but got an empty array instead.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'empty_array'}}

In [ ]:
### You can do same for reject and edit as well...